In [2]:
from elasticsearch7 import Elasticsearch
import numpy as np
import pandas as pd
import json

In [3]:
es = Elasticsearch(hosts="http://localhost:9200")

In [4]:
response = es.search(index="fluentd-k8s",)
hit_count = response["hits"]["total"]["value"]

In [5]:
response = es.search(index="fluentd-k8s", size=hit_count)

In [6]:
docs = response["hits"]["hits"]

In [7]:
docs = response["hits"]["hits"]
fields = {}
for num, doc in enumerate(docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        if key == "log":
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])
    # do stuff here!

In [8]:
df : pd.DataFrame = pd.DataFrame(fields)

In [9]:
log_df : pd.DataFrame = pd.json_normalize(df["log"].apply(json.loads))

In [10]:
tilt_df : pd.DataFrame = pd.json_normalize(
    log_df["tilt"]
    .apply(lambda x: x.replace("\'","\""))
    .apply(json.loads))\
        .add_prefix("tilt:")

In [65]:
combined_df = pd.concat([log_df,tilt_df],axis=1).drop("tilt",axis=1)
combined_df.to_csv("tilt-enhanced-event-log.csv",sep=",")

combined_df.index.name = "ocel:eid"
combined_df["ocel:timestamp"] = combined_df["timestamp"].apply(lambda x: x.replace(","," "))
combined_df["ocel:activity"]= combined_df["concept:name"]
combined_df.reset_index().to_json("tilt-enhanced-event-log.json",orient="records",)

In [40]:
with open("tilt-enhanced-event-log.json") as f:
    jo = json.load(f)

In [63]:
ocel = {"ocel:events":{},"ocel:objects":{},"ocel:global-log": {
    "ocel:attribute-names": [
      "label",
      "purpose",
      "basis"
    ],
    "ocel:object-types": [
      "tilt:data_disclosed",
      "tilt:purposes",
      "tilt:legal_bases"
    ],
    "ocel:version": "1.0",
    "ocel:ordering": "timestamp"
  },"ocel:global-event": {
    "ocel:activity": "__INVALID__"
  },
  "ocel:global-object": {
    "ocel:type": "__INVALID__"
  }}
for i in jo:
    ocel["ocel:events"][i["ocel:eid"]] = i
    ocel["ocel:events"][i["ocel:eid"]]["ocel:vmap"] = {}
    ocel["ocel:events"][i["ocel:eid"]]["ocel:omap"] = []
    for t in [k for k in i.keys() if k.startswith("tilt")]:
        ocel["ocel:events"][i["ocel:eid"]]["ocel:vmap"][t] = i[t]
    for j in i["tilt:data_disclosed"]:
        ocel["ocel:events"][i["ocel:eid"]]["ocel:omap"].append(j)
        if not j in ocel["ocel:objects"].keys():
            ocel["ocel:objects"][j]={
                "ocel:type": "tilt:data_disclosed",
                "ocel:ovmap":{"label":j}}
    for j in i["tilt:purposes"]:
        ocel["ocel:events"][i["ocel:eid"]]["ocel:omap"].append(j)
        if not j in ocel["ocel:objects"].keys():
            ocel["ocel:objects"][j]={
                "ocel:type": "tilt:purposes",
                "ocel:ovmap":{"purpose":j}}
    for j in i["tilt:legal_bases"]:
        ocel["ocel:events"][i["ocel:eid"]]["ocel:omap"].append(j)
        if not j in ocel["ocel:objects"].keys():
            ocel["ocel:objects"][j]={
                "ocel:type": "tilt:legal_bases",
                "ocel:ovmap":{"basis":j}}

In [64]:
with open('ocel.json', 'w') as f:
    json.dump(ocel, f)